# SpaceX Falcon 9 First Stage Landing Prediction
## Part 6: Data Visualization with Folium
***
### Objectives
*   Mark all launch sites on a map
*   Mark the success/failed launches for each site on the map
*   Calculate the distances between a launch site to its proximities
***
### Import Libraries

In [1]:
!pip3 install folium
!pip3 install wget
!pip install fontawesome

In [2]:
import folium
import wget
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

### Create Folium Map
First, let's try to add each launch site's location on a map using site's latitude and longitude coordinates.

In [3]:
spacex_df=pd.read_csv('03_dataset.csv')

Now, let's take a look at the coordinates for each site.

In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


Let's start populating our map by creating a folium Map object with an initial center location of the NASA Johnson Space Center at Houston, Texas.

In [5]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400',
                        fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(nasa_coordinate, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC'))
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in dataframe `launch_sites`.


In [6]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
for index, row in launch_sites_df.iterrows():
    
    coord = [row[1], row[2]]
    site = str(row[0])
    
    site_circle = folium.Circle(coord,radius=1000, color='#0000FF', fill=True).add_child(folium.Popup(site))
    site_marker = folium.map.Marker(coord, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                                                   html='<div style="font-size: 12; color:#0000FF;"><b>%s</b></div>' % site,))
    
    site_map.add_child(site_circle)
    site_map.add_child(site_marker)

site_map

We can see that all launch sites are in very close proximity to the coast and are in the southern-most parts of the west and east coasts. 

Next, let's try to enhance the map by adding the launch outcomes for each site and see which sites have high success rates.

In [7]:
marker_cluster = MarkerCluster().add_to(site_map)

# Add marker_color column to dataset based on mission success/failure
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['Class'].apply(assign_marker_color)
spacex_df.tail(10)

,LaunchSite,Latitude,Longitude,Class,marker_color
102,KSC LC 39A,28.608058,-80.603956,1,green
103,CCSFS SLC 40,28.561857,-80.577366,1,green
104,CCSFS SLC 40,28.561857,-80.577366,1,green
105,KSC LC 39A,28.608058,-80.603956,1,green
106,CCSFS SLC 40,28.561857,-80.577366,1,green
107,KSC LC 39A,28.608058,-80.603956,1,green
108,CCSFS SLC 40,28.561857,-80.577366,1,green
109,CCSFS SLC 40,28.561857,-80.577366,1,green
110,KSC LC 39A,28.608058,-80.603956,1,green
111,CCSFS SLC 40,28.561857,-80.577366,1,green


In [8]:
# Add marker_cluster to site_map
for index, record in spacex_df.iterrows():
    folium.Marker(location=[record.loc['Latitude'], record.loc['Longitude']], 
                  icon=folium.Icon(color='white', 
                    icon_color=record.loc['marker_color'])).add_to(marker_cluster)
site_map

From the color-labeled markers in marker clusters we can easily identify which launch sites have relatively high success rates.

Next, we need to explore and analyze the proximities of launch sites.


In [9]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter)

site_map.add_child(mouse_position)
site_map

In [10]:
from math import sin, cos, sqrt, atan2, radians

# Define function to calculate distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [11]:
import fontawesome as fa

# Display distances from launch sites to nearest coast 
launch_sites_df['Coast Lat'] = [28.56293, 28.61233, 34.63628]
launch_sites_df['Coast Long'] = [-80.56785, -80.59776, -120.62503]

for index, row in launch_sites_df.iterrows():
    
    llat = row.loc['Latitude']
    llong = row.loc['Longitude']
    clat = row.loc['Coast Lat']
    clong = row.loc['Coast Long']
    distance_coastline = calculate_distance(llat, llong, clat, clong)
    folium.Marker(location = [clat, clong], 
                  icon = folium.Icon(color='blue',
                                     icon_color='white',
                                     icon='tint', 
                                     prefix='fa'
                                    ),
                  popup = folium.Popup(html='<div style="font-size: 12; color: #d35400;">%s</div>' % "{:10.2f} KM".format(distance_coastline))
                 ).add_to(marker_cluster)
    
    folium.PolyLine(locations=[[llat, llong], [clat, clong]], weight=1).add_to(site_map)
site_map


In [12]:
# Function that maps POIs based on coordinates and marker/icon properties 
def add_POIs(latitudes, longitudes, marker_color, icon_color, icon_name, icon_prefix, line_color, launch_site_nos):
    for i, site in enumerate(launch_site_nos):
        l_lat = launch_sites_df.loc[site].loc['Latitude']
        l_long = launch_sites_df.loc[site].loc['Longitude']
        poi_lat = latitudes[i]
        poi_long = longitudes[i]
        distance = calculate_distance(l_lat, l_long, poi_lat, poi_long)
        folium.Marker(location = [poi_lat, poi_long], 
                      icon = folium.Icon(color=marker_color,
                                         icon_color=icon_color,
                                         icon=icon_name, 
                                         prefix=icon_prefix
                                        ),
                      popup = folium.Popup(html='<div style="font-size: 12; color: #d35400;">%s</div>' % "{:10.2f} KM".format(distance))
                     ).add_to(marker_cluster)
    
        folium.PolyLine(locations=[[l_lat, l_long], [poi_lat, poi_long]], weight=1, color=line_color).add_to(site_map)

In [13]:
# Add nearest cities
cities_lat = [34.63996, 28.61031, 28.40018]
cities_long = [-120.45822, -80.81131, -80.60532]
cities_ls = [2, 1, 0]
add_POIs(cities_lat, cities_long, 'purple', 'white', 'building', 'fa', 'purple', cities_ls)

# Add nearest airports
ap_lat = [28.51687, 28.51687, 34.6679]
ap_long = [-80.80032, -80.80032, -120.46509]
ap_ls = [0, 1, 2]
add_POIs(ap_lat, ap_long, 'green', 'white', 'plane', 'fa', 'green', ap_ls)
site_map

Based on the relationship between POIs and launch sites, it appears that launch sites are ~14km or further from airports and cities. 